# Phase 10: Generative Models
## Часть 1: Variational Autoencoders (VAE)

### В этом ноутбуке:

1. **Autoencoders** - сжатие и восстановление
2. **VAE** - вероятностный подход
3. **Reparametrization Trick** - обучение через sampling
4. **Latent Space** - исследование пространства
5. **Генерация новых образцов**

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import warnings
warnings.filterwarnings('ignore')

torch.manual_seed(42)
np.random.seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Device: {device}')

## 1. Синтетические данные

Создаём простой датасет изображений 8x8 с паттернами.

In [ ]:
def generate_patterns(n_samples=1000):
    """Генерация простых паттернов 8x8"""
    patterns = []
    
    for _ in range(n_samples):
        img = np.zeros((8, 8))
        pattern_type = np.random.randint(4)
        
        if pattern_type == 0:  # Горизонтальная линия
            row = np.random.randint(1, 7)
            img[row, 1:7] = 1
        elif pattern_type == 1:  # Вертикальная линия
            col = np.random.randint(1, 7)
            img[1:7, col] = 1
        elif pattern_type == 2:  # Квадрат
            size = np.random.randint(2, 4)
            start = np.random.randint(1, 6-size)
            img[start:start+size, start:start+size] = 1
        else:  # Диагональ
            for i in range(6):
                img[i+1, i+1] = 1
        
        # Добавляем шум
        img += np.random.normal(0, 0.1, (8, 8))
        img = np.clip(img, 0, 1)
        patterns.append(img)
    
    return np.array(patterns)

# Генерация данных
X = generate_patterns(2000)
X = torch.FloatTensor(X).view(-1, 1, 8, 8)

# Датасет
dataset = TensorDataset(X)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

print(f'Data shape: {X.shape}')

# Визуализация
fig, axes = plt.subplots(2, 8, figsize=(12, 3))
for i in range(16):
    ax = axes[i//8, i%8]
    ax.imshow(X[i, 0], cmap='gray')
    ax.axis('off')
plt.suptitle('Sample Patterns')
plt.show()

## 2. Обычный Autoencoder

Для сравнения сначала реализуем простой autoencoder.

In [ ]:
class Autoencoder(nn.Module):
    """Простой Autoencoder"""
    
    def __init__(self, latent_dim=8):
        super().__init__()
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, latent_dim)
        )
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.Sigmoid()
        )
    
    def forward(self, x):
        z = self.encoder(x)
        x_recon = self.decoder(z)
        return x_recon.view(-1, 1, 8, 8)
    
    def encode(self, x):
        return self.encoder(x)

# Тест
ae = Autoencoder(latent_dim=8)
test_input = torch.randn(1, 1, 8, 8)
output = ae(test_input)
print(f'Input: {test_input.shape} -> Output: {output.shape}')

## 3. Variational Autoencoder (VAE)

### Ключевые идеи:

1. Encoder выдаёт **распределение** (μ, σ), а не точку
2. **Reparametrization trick**: z = μ + σ * ε, где ε ~ N(0, 1)
3. **ELBO Loss** = Reconstruction + KL Divergence

In [ ]:
class VAE(nn.Module):
    """Variational Autoencoder"""
    
    def __init__(self, latent_dim=8):
        super().__init__()
        self.latent_dim = latent_dim
        
        # Encoder
        self.encoder = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU()
        )
        
        # Latent space parameters
        self.fc_mu = nn.Linear(16, latent_dim)
        self.fc_logvar = nn.Linear(16, latent_dim)
        
        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 32),
            nn.ReLU(),
            nn.Linear(32, 64),
            nn.Sigmoid()
        )
    
    def encode(self, x):
        h = self.encoder(x)
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        """Reparametrization trick"""
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + eps * std
    
    def decode(self, z):
        return self.decoder(z).view(-1, 1, 8, 8)
    
    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        x_recon = self.decode(z)
        return x_recon, mu, logvar

# Тест
vae = VAE(latent_dim=8)
test_input = torch.randn(1, 1, 8, 8)
recon, mu, logvar = vae(test_input)
print(f'Reconstruction: {recon.shape}')
print(f'Mu: {mu.shape}, LogVar: {logvar.shape}')

## 4. VAE Loss Function

**ELBO = Reconstruction Loss + KL Divergence**

$$\mathcal{L} = \mathbb{E}[\log p(x|z)] - D_{KL}(q(z|x) || p(z))$$

Для Gaussian prior:

$$D_{KL} = -\frac{1}{2} \sum (1 + \log\sigma^2 - \mu^2 - \sigma^2)$$

In [ ]:
def vae_loss(recon_x, x, mu, logvar, beta=1.0):
    """
    VAE Loss = Reconstruction + beta * KL Divergence
    
    beta-VAE: beta > 1 для лучшего disentanglement
    """
    # Reconstruction loss (Binary Cross Entropy)
    recon_loss = F.binary_cross_entropy(recon_x, x, reduction='sum')
    
    # KL Divergence
    kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    
    return recon_loss + beta * kl_loss, recon_loss, kl_loss

print('VAE Loss function готова')

## 5. Обучение VAE

In [ ]:
def train_vae(model, dataloader, epochs=100, lr=1e-3, beta=1.0):
    """Обучение VAE"""
    
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    history = {'total': [], 'recon': [], 'kl': []}
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_recon = 0
        total_kl = 0
        
        for batch in dataloader:
            x = batch[0].to(device)
            
            optimizer.zero_grad()
            recon, mu, logvar = model(x)
            
            loss, recon_loss, kl_loss = vae_loss(recon, x, mu, logvar, beta)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
            total_recon += recon_loss.item()
            total_kl += kl_loss.item()
        
        n = len(dataloader.dataset)
        history['total'].append(total_loss / n)
        history['recon'].append(total_recon / n)
        history['kl'].append(total_kl / n)
        
        if (epoch + 1) % 20 == 0:
            print(f'Epoch {epoch+1}, Loss: {total_loss/n:.4f}, '
                  f'Recon: {total_recon/n:.4f}, KL: {total_kl/n:.4f}')
    
    return history

# Обучение
vae = VAE(latent_dim=8)
print('Обучение VAE...\n')
history = train_vae(vae, dataloader, epochs=100, lr=1e-3, beta=1.0)

In [ ]:
# Визуализация обучения
fig, axes = plt.subplots(1, 3, figsize=(14, 3))

axes[0].plot(history['total'])
axes[0].set_title('Total Loss')
axes[0].set_xlabel('Epoch')

axes[1].plot(history['recon'])
axes[1].set_title('Reconstruction Loss')
axes[1].set_xlabel('Epoch')

axes[2].plot(history['kl'])
axes[2].set_title('KL Divergence')
axes[2].set_xlabel('Epoch')

plt.tight_layout()
plt.show()

## 6. Реконструкция

In [ ]:
# Тестируем реконструкцию
vae.eval()
with torch.no_grad():
    test_samples = X[:8].to(device)
    recon, _, _ = vae(test_samples)

# Визуализация
fig, axes = plt.subplots(2, 8, figsize=(12, 3))

for i in range(8):
    # Original
    axes[0, i].imshow(test_samples[i, 0].cpu(), cmap='gray')
    axes[0, i].axis('off')
    if i == 0:
        axes[0, i].set_title('Original')
    
    # Reconstruction
    axes[1, i].imshow(recon[i, 0].cpu(), cmap='gray')
    axes[1, i].axis('off')
    if i == 0:
        axes[1, i].set_title('Reconstructed')

plt.tight_layout()
plt.show()

## 7. Генерация новых образцов

In [ ]:
# Генерация из случайного z
vae.eval()
with torch.no_grad():
    # Сэмплируем из стандартного нормального распределения
    z = torch.randn(16, 8).to(device)
    generated = vae.decode(z)

# Визуализация
fig, axes = plt.subplots(2, 8, figsize=(12, 3))
for i in range(16):
    ax = axes[i//8, i%8]
    ax.imshow(generated[i, 0].cpu(), cmap='gray')
    ax.axis('off')

plt.suptitle('Generated Samples from Random z')
plt.show()

## 8. Исследование Latent Space

In [ ]:
# Интерполяция между двумя точками
vae.eval()
with torch.no_grad():
    # Берём два образца
    x1 = X[0:1].to(device)
    x2 = X[100:101].to(device)
    
    # Кодируем
    mu1, _ = vae.encode(x1)
    mu2, _ = vae.encode(x2)
    
    # Интерполяция
    n_steps = 8
    interpolations = []
    for alpha in np.linspace(0, 1, n_steps):
        z = (1 - alpha) * mu1 + alpha * mu2
        img = vae.decode(z)
        interpolations.append(img[0, 0].cpu())

# Визуализация
fig, axes = plt.subplots(1, n_steps, figsize=(12, 2))
for i, img in enumerate(interpolations):
    axes[i].imshow(img, cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(f'α={i/(n_steps-1):.1f}')

plt.suptitle('Latent Space Interpolation')
plt.show()

In [ ]:
# Визуализация 2D latent space (используя первые 2 компоненты)
vae.eval()
with torch.no_grad():
    mu, _ = vae.encode(X[:500].to(device))
    mu = mu.cpu().numpy()

plt.figure(figsize=(8, 6))
plt.scatter(mu[:, 0], mu[:, 1], alpha=0.5, s=10)
plt.xlabel('z[0]')
plt.ylabel('z[1]')
plt.title('Latent Space (first 2 dimensions)')
plt.colorbar(label='Sample index')
plt.show()

## Итоги

### Что мы изучили:

1. **Autoencoder** - сжатие и восстановление
2. **VAE** - вероятностный latent space
3. **Reparametrization trick** - градиенты через sampling
4. **ELBO Loss** - reconstruction + KL divergence

### Ключевые формулы:

**Reparametrization:**
$$z = \mu + \sigma \cdot \epsilon, \quad \epsilon \sim \mathcal{N}(0, I)$$

**KL Divergence (Gaussian):**
$$D_{KL} = -\frac{1}{2} \sum (1 + \log\sigma^2 - \mu^2 - \sigma^2)$$

### Преимущества VAE:

- Структурированный latent space
- Возможность генерации
- Интерполяция между образцами

### Следующий шаг:

В ноутбуке 02 изучим GAN - Generative Adversarial Networks.